In [2]:
import pandas as pd
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [3]:
pip install autograd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 901.9 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 4.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import autograd
from autograd import numpy as np


In [5]:
coordinates_and_disp = pd.read_csv("10\coordinates_and_disp.csv")

FileNotFoundError: [Errno 2] No such file or directory: '10\\coordinates_and_disp.csv'

In [6]:
coordinates_and_disp

,Node_no.,x_coor,y_coor,u_x,u_y,u_x_low,u_y_low,u_x_high,u_y_high
0,1,0.000000,0.000000,2.040851e-19,0.000000,-0.000019,2.309680e-07,-0.000130,0.000275
1,2,1.000000,0.000000,5.000000e-02,0.000000,0.050001,1.166761e-05,0.050068,0.000056
2,3,1.000000,1.000000,5.000000e-02,0.100000,0.049999,1.000012e-01,0.049944,0.100027
3,4,0.000000,1.000000,0.000000e+00,0.100000,0.000006,1.000041e-01,0.000166,0.099995
4,5,0.029412,0.000000,1.470588e-03,0.000000,0.001452,3.539026e-08,0.001353,0.000264
...,...,...,...,...,...,...,...,...,...
1433,1434,0.938818,0.041254,4.694088e-02,0.004125,0.046941,4.135551e-03,0.046992,0.004166
1434,1435,0.059067,0.041099,2.953350e-03,0.004110,0.002937,4.110577e-03,0.002845,0.004335
1435,1436,0.979437,0.610897,4.897184e-02,0.061090,0.048974,6.109863e-02,0.048885,0.061025
1436,1437,0.959345,0.498258,4.796725e-02,0.049826,0.047970,4.983556e-02,0.047894,0.049759


In [10]:
reactions = pd.read_csv("10//reactions.csv")

In [12]:
reactions.head()

,reactions
0,-0.510058
1,0.510058
2,-0.575649
3,0.575649


In [ ]:
coordinates_and_disp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1438 entries, 0 to 1437
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Node_no.  1438 non-null   int64  
 1   x_coor    1438 non-null   float64
 2   y_coor    1438 non-null   float64
 3   u_x       1438 non-null   float64
 4   u_y       1438 non-null   float64
 5   u_x_low   1438 non-null   float64
 6   u_y_low   1438 non-null   float64
 7   u_x_high  1438 non-null   float64
 8   u_y_high  1438 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 101.2 KB


In [11]:
reactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   reactions  4 non-null      float64
dtypes: float64(1)
memory usage: 160.0 bytes


In [13]:
import tensorflow as tf
import numpy as np

In [35]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(6, 16)  # First hidden layer
        self.fc2 = nn.Linear(16, 8)  # Second hidden layer
        self.fc3 = nn.Linear(8, 2)   # Output layer

    def forward(self, x):
        # Apply ReLU activation between hidden layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
model=SimpleNN()


# y=model(X)


In [16]:
# HELPER Functions for course project

def computeJacobian(F):
    """
    Compute Jacobian from deformation gradient.

    _Input Arguments_

    - `F` - deformation gradient in Voigt notation

    _Output Arguments_

    - `J` - Jacobian

    ---

    """
    F11 = F[:,0:1]
    F12 = F[:,1:2]
    F21 = F[:,2:3]
    F22 = F[:,3:4]

    J = F11*F22 - F12*F21
    return J

In [17]:
def computeCauchyGreenStrain(F):
    """
    Compute right Cauchy-Green strain tensor from deformation gradient.

    _Input Arguments_

    - `F` - deformation gradient in Voigt notation

    _Output Arguments_

    - `C` - Cauchy-Green strain tensor in Voigt notation

    ---

    """
    F11 = F[:,0:1]
    F12 = F[:,1:2]
    F21 = F[:,2:3]
    F22 = F[:,3:4]

    C11 = F11**2 + F21**2
    C12 = F11*F12 + F21*F22
    C21 = F11*F12 + F21*F22
    C22 = F12**2 + F22**2

    C = torch.cat((C11,C12,C21,C22),dim=1)
    return C

In [18]:
def computeStrainInvariants(C):
    """
    Compute invariants of the Cauchy-Green strain tensor.
    Plane strain is assumed.

    _Input Arguments_

    - `C` - Cauchy-Green strain tensor in Voigt notation

    _Output Arguments_

    - `I1` - 1st invariant

    - `I2` - 2nd invariant

    - `I3` - 3rd invariant

    ---

    """
    C11 = C[:,0:1]
    C12 = C[:,1:2]
    C21 = C[:,2:3]
    C22 = C[:,3:4]

    I1 = C11 + C22 + 1.0
    I2 = C11 + C22 - C12*C21 + C11*C22
    I3 = C11*C22 - C12*C21
    return I1, I2, I3


In [19]:


def computeStrainInvariantDerivatives(F,i,secondDerivative=False):
    """
    Compute derivatives of the invariants of the Cauchy-Green strain tensor with respect to the deformation gradient.
    Plane strain is assumed.

    _Input Arguments_

    - `F` - deformation gradient in Voigt notation

    - `i` - specify the invariant that should be differentiated

    - `secondDerivative` - specify if second derivative should be computed

    _Output Arguments_

    - `dIdF` - derivative (note that the size of `dIdF` depends on the choice of `secondDerivative`)

    ---

    """
    F11 = F[:,0:1]
    F12 = F[:,1:2]
    F21 = F[:,2:3]
    F22 = F[:,3:4]
    if not(secondDerivative):
        dIdF = torch.zeros(F.shape[0],F.shape[1])
        if(i==1):
            # dI1/dF:
            dIdF = 2.0*F
        elif(i==2):
            # dI2/dF:
            dIdF11 = 2.0*F11 - 2.0*F12*F21*F22 + 2.0*F11*(F22**2)
            dIdF12 = 2.0*F12 + 2.0*F12*(F21**2) - 2.0*F11*F21*F22
            dIdF21 = 2.0*F21 + 2.0*(F12**2)*F21 - 2.0*F11*F12*F22
            dIdF22 = 2.0*F22 - 2.0*F11*F12*F21 + 2.0*(F11**2)*F22
            dIdF = torch.cat((dIdF11,dIdF12,dIdF21,dIdF22),dim=1)
        elif(i==3):
            # dI3/dF:
            J = F11*F22 - F12*F21
            dIdF11 = 2.0*F22 * J
            dIdF12 = -2.0*F21 * J
            dIdF21 = -2.0*F12 * J
            dIdF22 = 2.0*F11 * J
            dIdF = torch.cat((dIdF11,dIdF12,dIdF21,dIdF22),dim=1)
        else:
            raise ValueError('Incorrect invariant index')
    if secondDerivative:
        dIdF = torch.zeros(F.shape[1],F.shape[1])
        if(i==1):
            # d(dI1/dF)/dF:
            dIdF = 2.0*torch.eye(F.shape[1])
        elif(i==3):
            # d(dI3/dF)/dF:
            J = F11*F22 - F12*F21
            dJdF11 = F22
            dJdF12 = - F21
            dJdF21 = - F12
            dJdF22 = F11
            # d(dI3/dF)/dF11:
            dIdF[0,0] = 2.0 * F22 * dJdF11
            dIdF[0,1] = -2.0 * F21 * dJdF11
            dIdF[0,2] = -2.0 * F12 * dJdF11
            dIdF[0,3] = 2.0 * J + 2.0 * F11 * dJdF11
            # d(dI3/dF)/dF12:
            dIdF[1,0] = 2.0 * F22 * dJdF12
            dIdF[1,1] = -2.0 * F21 * dJdF12
            dIdF[1,2] = -2.0 * J -2.0 * F12 * dJdF12
            dIdF[1,3] = 2.0 * F11 * dJdF12
            # d(dI3/dF)/dF21:
            dIdF[2,0] = 2.0 * F22 * dJdF21
            dIdF[2,1] = -2.0 * J + -2.0 * F21 * dJdF21
            dIdF[2,2] = -2.0 * F12 * dJdF21
            dIdF[2,3] = 2.0 * F11 * dJdF21
            # d(dI3/dF)/dF22:
            dIdF[3,0] = 2.0 * J + 2.0 * F22 * dJdF22
            dIdF[3,1] = -2.0 * F21 * dJdF22
            dIdF[3,2] = -2.0 * F12 * dJdF22
            dIdF[3,3] = 2.0 * F11 * dJdF22
        else:
            raise ValueError('Incorrect invariant index')
    return dIdF


In [21]:

def computeFeatures(I1, I2, I3):
    """
    Compute the features dependent on the right Cauchy-Green strain invariants.
    Note that the features only depend on I1 and I3.

    _Input Arguments_

    - `I1` - 1st invariant

    - `I2` - 2nd invariant

    - `I3` - 3rd invariant

    _Output Arguments_

    - `x` - features

    ---

    """
    #Generalized Mooney-Rivlin.
    #The Gent-Thomas model cannot be represented by the generalized
    #Mooney-Rivlin model. An additional feature has to be added.
    considerGentThomas = True
    #Polynomial terms degree.
    Na = 7
    #Volumetric terms degree.
    Nb = 7

    K1 = I1 * torch.pow(I3,-1/3) - 3.0
    K2 = (I1 + I3 - 1) * torch.pow(I3,-2/3) - 3.0
    J = torch.sqrt(I3)
    #Calculate the number of features.
    numFeatures = 0
    #Polynomial terms (dependent on K1 and K2).
    for n in range(Na):
        numFeatures += n + 2
    #Volumetric terms (dependent on J).
    for m in range(Nb):
        numFeatures += 1
    #Additional Gent-Thomas feature.
    if considerGentThomas:
        numFeatures += 1
    #Calculate the features.
    x = torch.zeros(I1.shape[0],numFeatures)
    i=-1;
    #Polynomial terms (dependent on K1 and K2).
    for p in range(1,Na+1):
        for q in range(p+1):
            i+=1; x[:,i:(i+1)] = K1**(p-q) * K2**q

    #Volumetric terms (dependent on J):
    for m in range(1,Nb+1):
        i+=1; x[:,i:(i+1)] = (J-1)**(2*m)
    #Additional Gent-Thomas feature.
    if considerGentThomas:
        i+=1; x[:,i:(i+1)] = torch.log((K2+3.0)/3.0)

    return x



In [22]:
def getNumberOfFeatures():
    """
    Compute number of features.

    _Input Arguments_

    - _none_

    _Output Arguments_

    - `features.shape[1]` - number of features

    ---

    """
    features = computeFeatures(torch.zeros(1,1),torch.zeros(1,1),torch.zeros(1,1))
    return features.shape[1]


In [23]:
X = torch.tensor(
    [
        [1.0, 2.0, 3.0, 4.0, 5.0, 6.0],
        [6.0, 5.0, 4.0, 3.0, 2.0, 1.0],
        [1.0, 3.0, 5.0, 7.0, 9.0, 11.0],
        [2.0, 4.0, 6.0, 8.0, 10.0, 12.0],
        [2.0, 4.0, 6.0, 8.0, 10.0, 12.0]
    ],
    requires_grad=True,
)

In [75]:
grad_u = autograd.functional.jacobian(model,X, create_graph=True)

In [61]:
grad_u[1]

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [-0.0061,  0.0021, -0.0286, -0.0243,  0.0228, -0.0052],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0026,  0.0273,  0.0109, -0.0208, -0.0418, -0.0192],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]])

In [77]:
I = torch.eye(4)
F_X = I + grad_u_final

In [78]:
F_X

tensor([[ 0.9986,  0.0161, -0.0198, -0.0267],
        [-0.0061,  1.0021,  0.0026,  0.0273],
        [-0.0011,  0.0173,  0.9843, -0.0113],
        [-0.0014,  0.0161, -0.0198,  0.9733]])

In [79]:
C_x = computeCauchyGreenStrain(F_X)

In [80]:
C_x

tensor([[ 9.9750e-01,  1.6631e-02,  1.6631e-02,  9.7130e-04],
        [ 4.4330e-05, -6.0716e-03, -6.0716e-03,  1.0050e+00],
        [ 9.6884e-01, -1.1100e-02, -1.1100e-02,  4.2546e-04],
        [ 3.9374e-04, -1.9286e-02, -1.9286e-02,  9.4763e-01]])

In [81]:
I1, I2, I3 = computeStrainInvariants(C_x)

In [82]:
I1, I2, I3

(tensor([[1.9985],
         [2.0050],
         [1.9693],
         [1.9480]]),
 tensor([[0.9992],
         [1.0050],
         [0.9696],
         [0.9480]]),
 tensor([[6.9228e-04],
         [7.6857e-06],
         [2.8899e-04],
         [1.1824e-06]]))

In [84]:
all_features = computeFeatures(I1, I2, I3)
all_features.shape

torch.Size([4, 43])

In [85]:
all_Invariants = [I1, I2, I3]

In [ ]:
def find_P(all_features , all_Invariants,beta , F):
    for i in range(all_Invariants):
        